In [2]:
import pandas as pd
import requests
import xml.etree.ElementTree as ET

In [3]:
def read_dataframe(year, field):
    
    '''
    Read the dataframe of a year
    Args: filed (disease/gene/demo/pm), year: a single year
    Returns: a dataframe
    '''
    
    columns = ["trec_topic_number", "trec_doc_id", "year"]
    
    if field == "disease":
        columns.append("disease_desc")
    elif field == "pm":
        columns.append("pm_rel_desc")
    elif field == "demo":
        columns.append("demographics_desc")
    elif field == "gene":
        columns.append("gene1_annotation_desc")
        columns.append("gene2_annotation_desc")
        columns.append("gene3_annotation_desc")
        
    df = pd.read_csv("/Users/jiamingqu/Desktop/2017.2018/data/rawjudgements/judgments"+str(year)+".csv")
    df['year'] = year
    
    return df[columns]

In [9]:
def searching_by_docid(docid):
    query_string = "pmid:" + str(docid)
    payload = {'q': query_string, 'start': '0', 'rows': 1}
    response = requests.get('http://localhost:12345/solr/test_medline/select', params = payload)
    return response.json()["response"]["docs"][0]

In [10]:
def read_query_topics(year, field):
    
    '''
    Read the query topics of a certain year
    Args: year: a certain year
          field (disease/gene/demo/pm)
    Returns: a <topic_no, query> dictionary
    '''
    
    tree = ET.parse("/Users/jiamingqu/Desktop/2017.2018/data/topics/topics" + str(year) + ".xml")
    root = tree.getroot()
    
    results = {}
    count = 1
    
    for child in root:
        if field == "disease":
            results[count] = child[0].text
            count += 1
        elif field == "gene":
            results[count] = child[1].text
            count += 1
        elif field == "demo":
            results[count] = child[2].text
            count += 1
            
    return results